In [1]:
import anndata as ad

# PATH = '/home/ec2-user/cell_census/tabula_sapiens__sample_single_cell__label_cell_type__processed.h5ad'
PATH = '/home/ec2-user/cell_census/tabula_sapiens__sample_donor_id__label_cell_type.h5ad'

adata = ad.read_h5ad(PATH)

from enformer_pytorch import GenomeIntervalDataset

class MyGenomeIntervalDataset(GenomeIntervalDataset):
    def __init__(self, **kwargs):
        super(MyGenomeIntervalDataset, self).__init__(**kwargs)

    def __getitem__(self, ind):
        item = super().__getitem__(ind)
        label = self.df.row(ind)[4]
        return label, item


import torch
import polars as pl
import zarr
from enformer_pytorch import Enformer
from torch.utils.data import DataLoader
from tqdm import tqdm
import pyfaidx
from pathlib import Path

torch.multiprocessing.freeze_support()

BASE_PT = "/home/ec2-user/enformer"
DEVICE = "cuda:0"

# BASE_PT = "/Users/nsofroniew/Documents/data/multiomics/enformer"
# DEVICE = "cpu"

FASTA_PT = BASE_PT + "/Homo_sapiens.GRCh38.dna.toplevel.fa"
GENE_INTERVALS_PT = BASE_PT + "/Homo_sapiens.GRCh38.genes.bed"
GENE_INTERVALS_VAR_PT = BASE_PT + "/Homo_sapiens.GRCh38.genes.clinvar.vcf.bed"
EMBEDDING_PT = BASE_PT + "/Homo_sapiens.GRCh38.genes.enformer_embeddings.zarr"
EMBEDDING_PT_TSS = BASE_PT + "/Homo_sapiens.GRCh38.genes.enformer_embeddings_tss.zarr"
MODEL_PT = "EleutherAI/enformer-official-rough"

def filter_df_fn(df, name="column_5"):
    return df.filter(pl.col(name).is_in(list(adata.var_names)))

print("Converting fasta file")
pyfaidx.Faidx(FASTA_PT)
print("Fasta file done")

model = Enformer.from_pretrained(MODEL_PT, output_heads=dict(human = 5313), use_checkpointing = False)
model.to(DEVICE)

Converting fasta file
Fasta file done


Some weights of the model checkpoint at EleutherAI/enformer-official-rough were not used when initializing Enformer: ['_heads.mouse.0.weight', '_heads.mouse.0.bias']
- This IS expected if you are initializing Enformer from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Enformer from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Enformer(
  (stem): Sequential(
    (0): Conv1d(4, 768, kernel_size=(15,), stride=(1,), padding=(7,))
    (1): Residual(
      (fn): Sequential(
        (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): GELU()
        (2): Conv1d(768, 768, kernel_size=(1,), stride=(1,))
      )
    )
    (2): AttentionPool(
      (pool_fn): Rearrange('b d (n p) -> b d n p', p=2)
      (to_attn_logits): Conv2d(768, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
  )
  (conv_tower): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): GELU()
        (2): Conv1d(768, 768, kernel_size=(5,), stride=(1,), padding=(2,))
      )
      (1): Residual(
        (fn): Sequential(
          (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): GELU()
          (2): Conv1d(768, 768, kernel_size=(1,), stri

In [2]:
def filter_df_fn(df, name="column_5"):
    return df.filter(pl.col(name).is_in(list(adata.var_names)))

In [3]:
import polars as pl

df = pl.read_csv(GENE_INTERVALS_VAR_PT, separator='\t', has_header=False)
df = filter_df_fn(df)

In [4]:
from enformer_pytorch.data import str_to_one_hot


class MyGenomeIntervalVariantDataset(GenomeIntervalDataset):
    def __init__(self, **kwargs):
        super(MyGenomeIntervalVariantDataset, self).__init__(**kwargs)

    def __getitem__(self, ind):
        item = super().__getitem__(ind)
        label = self.df.row(ind)[4]
        start = self.df.row(ind)[1]
        pos = self.df.row(ind)[5]
        ref = self.df.row(ind)[6]
        alt = self.df.row(ind)[7]
        loc = pos - start - 1
        assert abs(item[loc] - str_to_one_hot(ref)).max() == 0.0 # confirm ref match
        return label, item, loc, str_to_one_hot(alt)

In [5]:
len(df)

143190

In [6]:
ds = MyGenomeIntervalVariantDataset(
    bed_file=GENE_INTERVALS_VAR_PT,  # bed file - columns 0, 1, 2 must be <chromosome>, <start position>, <end position>
    fasta_file=FASTA_PT,  # path to fasta file
    return_seq_indices=False,  # return nucleotide indices (ACGTN) or one hot encodings
    rc_aug=False,
    filter_df_fn=filter_df_fn,
)
dl = DataLoader(ds, batch_size=2, shuffle=False, num_workers=0) # type: DataLoader

# Create zarr files
SEQ_EMBED_DIM = 896
EMBED_DIM = 3072
NUM_GENES = len(ds)
TSS = int(SEQ_EMBED_DIM // 2)

paths = (Path(EMBEDDING_PT), Path(EMBEDDING_PT_TSS))

In [7]:
label, seq, pos, alt = ds[100]
# seq[pos] = alt

In [8]:
for d in ds:
    d[0]

In [ ]:
import pandas as pd
import numpy as np

# targets_txt = 'https://raw.githubusercontent.com/calico/basenji/0.5/manuscripts/cross2020/targets_human.txt'
# df_targets = pd.read_csv(targets_txt, sep='\t')
df_targets = pd.read_csv(BASE_PT + '/targets_human.txt')
df_targets.shape  # (5313, 8) With rows match output shape above.
cage_indices = np.where(df_targets['description'].str.startswith('CAGE:'))[0]

In [ ]:
import zarr

EMBEDDING_PT_TSS = BASE_PT + "/Homo_sapiens.GRCh38.genes.enformer_embeddings_inds.zarr"
z = zarr.open(EMBEDDING_PT_TSS)
genes = list(adata.var_names)
max_inds_dict = dict(zip(genes, list(z[:])))

In [ ]:
# z_embedding_full = zarr.open(
#     EMBEDDING_PT,
#     mode="w",
#     shape=(NUM_GENES, SEQ_EMBED_DIM, EMBED_DIM),
#     chunks=(1, SEQ_EMBED_DIM, EMBED_DIM),
#     dtype='float32',
# )

# z_embedding_tss = zarr.open(
#     EMBEDDING_PT_TSS,
#     mode="w",
#     shape=(NUM_GENES, EMBED_DIM),
#     chunks=(1, EMBED_DIM),
#     dtype='float32',
# )

# index = 0
# for labels, batch in tqdm(dl):
#     # calculate embedding
#     with torch.no_grad():
#         output, embeddings = model(batch.to(DEVICE), return_embeddings=True)
#         embeddings = embeddings.detach().cpu().numpy()

#     tss_embedding = embeddings[:, TSS]

#     # save full and reduced embeddings
#     batch_size = len(embeddings)
#     z_embedding_full[index : index + batch_size] = embeddings
#     z_embedding_tss[index : index + batch_size] = tss_embedding
#     index += batch_size

In [4]:
# label, seq, pos, alt = ds[0]
# seq[pos] = alt

In [5]:
labels, batch, pos, alt = next(iter(dl))

NameError: name 'dl' is not defined

In [ ]:
with torch.no_grad():
    output, embeddings = model(batch.to(DEVICE), return_embeddings=True)
    cage_expression = output['human'][:, :, cage_indices].mean(dim=-1)
    max_inds = torch.argmax(cage_expression, dim=-1)
    batch_size = len(embeddings)
    tss_embedding = embeddings[torch.arange(batch_size), max_inds]

    batch[torch.arange(batch.shape[0]), pos] = alt.squeeze(dim=1)
    output, embeddings = model(batch.to(DEVICE), return_embeddings=True)
    tss_embedding_var = embeddings[torch.arange(batch_size), max_inds]

In [ ]:
tss_embedding - tss_embedding_var

In [ ]:
tss_embedding.shape

In [ ]:
result = embeddings[torch.arange(2), :, max_inds]


In [ ]:
result.shape

In [ ]:
cage_expression.shape

In [ ]:
label, seq = ds[20]

In [ ]:
seq

In [ ]:
with torch.no_grad():
    output, embeddings = model(seq.to(DEVICE), return_embeddings=True)
    cage_expression = output['human'][:, cage_indices].mean(dim=1)
    max_ind = torch.argmax(cage_expression)
    tss_embedding = embeddings[max_ind]

In [ ]:
embeddings.shape

In [ ]:
list(ds.df['column_5'])

In [ ]:
max_ind = torch.argmax(cage_expression)

In [ ]:
tss_embedding.shape

In [ ]:
embeddings.shape

In [ ]:
output['human'].shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
cage_indices

In [ ]:
plt.plot([TSS, TSS], [0, 100]);
plt.plot(output['human'][:, cage_indices].mean(dim=1).detach().cpu());

In [ ]:
# !pip install pyensembl

In [ ]:
# from pyensembl import Genome

# gtf_file_path = BASE_PT + '/Homo_sapiens.GRCh38.77.gtf'

# genome = Genome(reference_name='GRCh38',
#     annotation_name='my_genome_features',
#     gtf_path_or_url=gtf_file_path)
# # genome.index()

In [ ]:
label

In [ ]:
from pyensembl import EnsemblRelease

genome = EnsemblRelease(77)

In [ ]:
gene = genome.gene_by_id(label)

# Observed and predicted gene expression values were obtained by summing up the observed/predicted CAGE read counts
# at all unique TSS locations of the gene. For each TSS location, we used the 128-bp bin overlapping the TSS as well
# as the two neighboring bins (3 bins in total).

# For each gene, look through all transcipts - protein coding / not, and record offsets from gene start
gene.transcripts

In [ ]:
from scipy.ndimage import gaussian_filter1d

def get_tss(gene_id, tss=TSS, length=SEQ_EMBED_DIM, sigma=8):
    gene = genome.gene_by_id(gene_id)
    starts = np.array([tss + np.round((ts.start - gene.start) / 128) for ts in gene.transcripts], dtype=int)
    starts = starts[starts>=0]
    starts = starts[starts<length]
    vector = np.zeros(length)
    vector[starts] = 1.0
    if sigma is not None:
        vector = gaussian_filter1d(vector, sigma)
    return vector

In [ ]:
plt.plot(get_tss(label))

In [ ]:
def get_tss_locations(genes, tss=TSS):
    locations = {}
    for gene_id in genes:
        gene = genome.gene_by_id(gene_id)
        start_diffs = [tss + np.round((ts.start - gene.start) / 128) for ts in gene.transcripts]
        locations[gene_id] = start_diffs
    return locations

In [ ]:
locations = get_tss_locations(list(ds.df['column_5']))

In [ ]:
locations

In [ ]:
gene = genome.gene_by_id(label)
start_diffs = [TSS + np.round((ts.start - gene.start) / 128) for ts in gene.transcripts]

In [ ]:
start_diffs

In [ ]:
for st in start_diffs:
    plt.plot([st, st], [0, 100], color='k');
# plt.plot(output['human'][:, cage_indices].mean(dim=1).cpu());
plt.ylim([0, 20])

In [ ]:
start_diffs

In [ ]:
def extract_embeddings(embeddings, cage_expression, tss_tensors, sigmas, tss=TSS):
    # Embeddings include
    #   TSS
    #   sum over all
    #   argmax over all
    #   sum over TSS
    #   argmax over TSS
    #   sum over TSS sigma 3, 8, 16
    #   argmax over TSS sigma 3, 8, 16
    batch_size = embeddings.shape[0]
    scaled_cage_expression = cage_expression * tss_tensors

    tss_emb = embeddings[:, TSS]
    sum_emb = embeddings.sum(dim=1)
    max_inds = torch.argmax(cage_expression, dim=-1)
    amax_emb = embeddings[torch.arange(batch_size), max_inds]
    
    max_inds = torch.argmax(scaled_cage_expression, dim=-1)
    amax_tss_emb = embeddings[torch.arange(batch_size), max_inds]

    max_inds = torch.argmax(scaled_cage_expression, dim=-1)
    sum_tss_emb = (embeddings * scaled_cage_expression).sum(dim=1)

    all_emb = [tss_emb, sum_emb, amax_emb, amax_tss_emb, sum_tss_emb]
    for sigma in sigmas:
        ks = 2 * int(sigma / 2 * 3)
        tss_tensors_conv = gaussian_filter_1d(tss_tensors, kernel_size=ks, sigma=sigma)
        scaled_cage_expression = cage_expression * tss_tensors_conv
        max_inds = torch.argmax(scaled_cage_expression, dim=-1)
        amax_tss_emb = embeddings[torch.arange(batch_size), max_inds]

        max_inds = torch.argmax(scaled_cage_expression, dim=-1)
        sum_tss_emb = (embeddings * scaled_cage_expression).sum(dim=1)

        all_emb.append(amax_tss_emb)
        all_emb.append(sum_tss_emb)

    return torch.stack(all_emb, dim=0) # 5 + 2 * len(sigmas)

In [ ]:
def extract_embeddings(embeddings, cage_expression, tss_tensors, sigmas, tss):
    # Embeddings include
    #   TSS -1, 0, 1
    #   argmax over TSS -1, 0, 1
    batch_size = embeddings.shape[0]
    len_seq = tss_tensors.shape[1] - 1
    scaled_cage_expression = cage_expression * tss_tensors

    tss_emb = embeddings[:, tss]
    tss_emb_m1 = embeddings[:, tss - 1]
    tss_emb_1 = embeddings[:, tss + 1]

    max_inds = torch.argmax(scaled_cage_expression, dim=-1)
    amax_emb = embeddings[torch.arange(batch_size), max_inds]
    amax_emb_m1 = embeddings[torch.arange(batch_size), torch.clip(max_inds - 1, 0, len_seq)]
    amax_emb_1 = embeddings[torch.arange(batch_size), torch.clip(max_inds + 1, 0, len_seq)]
    
    all_emb = [tss_emb, tss_emb_m1, tss_emb_1, amax_emb, amax_emb_m1, amax_emb_1]
    return torch.stack(all_emb, dim=0) # 6

In [ ]:
def extract_embeddings(embeddings, cage_expression, tss_tensors, sigmas, tss):
    # Embeddings include
    #   argmax over TSS -1, 0, 1 for top 5
    batch_size = embeddings.shape[0]
    len_seq = tss_tensors.shape[1] - 1
    scaled_cage_expression = cage_expression * tss_tensors

    # max_inds = torch.argmax(scaled_cage_expression, dim=-1)
    topk_inds, tok_values = torch.topk(scaled_cage_expression, 5, dim=-1)
    topk_inds[tok_values == 0] = tss

    all_emb = []
    for i in range(topk_inds.shape[1]):
        max_inds = topk_inds[:, i]
        amax_emb = embeddings[torch.arange(batch_size), max_inds]
        amax_emb_m1 = embeddings[torch.arange(batch_size), torch.clip(max_inds - 1, 0, len_seq)]
        amax_emb_1 = embeddings[torch.arange(batch_size), torch.clip(max_inds + 1, 0, len_seq)]
        all_emb += [amax_emb, amax_emb_m1, amax_emb_1]
    return torch.stack(all_emb, dim=0) # 15

In [ ]:
import torch

In [ ]:
A = torch.rand((2, 89))

In [ ]:
torch.topk(A, 5, dim=-1).indices[:, 0]

In [ ]:
torch.argmax(A, dim=-1)

In [ ]:
scaled_cage_expression = cage_expression * tss_tensors
max_inds = torch.argmax(cage_expression, dim=-1)
tss_embedding = embeddings[torch.arange(batch_size), max_inds]

In [ ]:
import anndata as ad


PATH = '/home/ec2-user/cell_census/tabula_sapiens__tss_max__sample_donor_id__label_cell_type.h5ad'

adata = ad.read_h5ad(PATH)

In [ ]:
adata.varm

In [ ]:
for name in list(adata.varm.keys()):
    del adata.varm[name]

In [ ]:
import zarr

z = zarr.open('/home/ec2-user/enformer/Homo_sapiens.GRCh38.genes.enformer_embeddings_5x_top10_pc_0.zarr', 'r')

In [ ]:
z.shape

In [ ]:
# import numpy as np
#     # Embeddings include
#     #   TSS
#     #   sum over all
#     #   argmax over all
#     #   argmax over TSS
#     #   sum over TSS
#     #   argmax over TSS sigma 3, 8, 16
#     #   sum over TSS sigma 3, 8, 16

# names = ['', '_sum', '_amax', '_tss_amax', '_tss_sum']
# sigmas = [3, 8, 16, 32, 64]
# for s in sigmas:
#     names.append('_tss_amax_' + str(s))
#     names.append('_tss_sum_' + str(s))

# for i, name in enumerate(names):
#     adata.varm['embedding' + name] = np.asarray(z[i])

In [ ]:
# import numpy as np

# names = ['', '_m1', '_1', '_amax', '_amax_m1', '_amax_1']

# for i, name in enumerate(names):
#     adata.varm['embedding' + name] = np.asarray(z[i])

In [ ]:
import numpy as np

k = 0
for i in range(10):
    for j in ['', '_m2', '_m1', '_1', '_2']:
        adata.varm['embedding_amax_' + str(i) + str(j)] = np.asarray(z[k])
        k += 1

In [ ]:
PATH_3 = '/home/ec2-user/cell_census/tabula_sapiens__5x_top10_pc__sample_donor_id__label_cell_type.h5ad'

adata.write_h5ad(PATH_3)

In [ ]:
1

In [ ]:
z[0, :2, :2]

In [ ]:
adata.varm

In [ ]:
list(adata.varm.keys())

In [ ]:
PATH_1 = '/home/ec2-user/cell_census/tabula_sapiens__all_pc__sample_donor_id__label_cell_type.h5ad'
PATH_2 = '/home/ec2-user/cell_census/tabula_sapiens__all__sample_donor_id__label_cell_type.h5ad'

adata_1 = ad.read_h5ad(PATH_1)
adata_2 = ad.read_h5ad(PATH_2)

In [ ]:
adata_1.varm['embedding_tss_amax'][:5,:5]

In [ ]:
adata_2.varm['embedding_tss_amax'][:5, :5]

In [ ]:
import numpy as np

In [ ]:
np.testing.assert_almost_equal(adata_1.varm['embedding_tss_amax'], adata_2.varm['embedding_tss_amax'])

In [ ]:
(adata_1.varm['embedding'] - adata_2.varm['embedding']).max()